In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 200)

In [2]:
#df_train = pd.read_parquet("../data/train.parquet")

In [8]:
#df_sample = df_train.sample(frac=0.01, random_state=42)
#df_sample.to_parquet("../data/train_sample_1pct.parquet")

In [10]:
df_train = pd.read_parquet("../data/train_sample_1pct.parquet")

In [11]:
print(df_train.columns.tolist())
df_train.head().T

['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']


,10478479,2240033,24966385,38721384,41464765
key,2010-01-16 20:06:00.000000168,2010-09-22 23:21:00.00000065,2013-10-17 18:40:14.0000001,2015-06-09 19:39:42.0000009,2012-05-01 07:16:59.0000001
fare_amount,3.3,6.5,5.0,40.5,7.7
pickup_datetime,2010-01-16 20:06:00 UTC,2010-09-22 23:21:00 UTC,2013-10-17 18:40:14 UTC,2015-06-09 19:39:42 UTC,2012-05-01 07:16:59 UTC
pickup_longitude,-73.953323,-73.985542,-73.999945,-73.974533,-73.952835
pickup_latitude,40.778532,40.738333,40.718097,40.756237,40.786351
dropoff_longitude,-73.953375,-74.00335,-73.999239,-73.923149,-73.974022
dropoff_latitude,40.772505,40.738935,40.723103,40.86721,40.762765
passenger_count,1,1,1,1,1


In [12]:
features = [
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count'
]


In [18]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

X = df_train_small[features]
y = df_train_small["fare_amount"]

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LGBMRegressor(n_estimators=1000, learning_rate=0.1, random_state=42)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
#        early_stopping_rounds=50,
#        verbose=100
    )

    y_pred = model.predict(X_val)
#    rmse = mean_squared_error(y_val, y_pred, squared=False)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)

print(f"平均RMSE: {np.mean(rmse_scores):.4f}")




[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001764 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1028
[LightGBM] [Info] Number of data points in the train set: 443391, number of used features: 5
[LightGBM] [Info] Start training from score 11.335547
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001832 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1027
[LightGBM] [Info] Number of data points in the train set: 443391, number of used features: 5
[LightGBM] [Info] Start training from score 11.331310
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001979 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

In [14]:
import lightgbm
print(lightgbm.__version__)

4.6.0


In [15]:
import sklearn
print(sklearn.__version__)


1.6.1


In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np

print("y_val type:", type(y_val))
print("y_pred type:", type(y_pred))

print("mean_squared_error function:", mean_squared_error)


y_val type: <class 'pandas.core.series.Series'>
y_pred type: <class 'numpy.ndarray'>
mean_squared_error function: <function mean_squared_error at 0x17b9d2170>


In [17]:
import inspect
print(inspect.getfile(mean_squared_error))


/opt/anaconda3/envs/new-york-taxi-fare-prediction/lib/python3.10/site-packages/sklearn/utils/_param_validation.py
